<a href="https://colab.research.google.com/github/elenaajayi/Computer-Vision-Fundamentals---CUS-754/blob/main/Capstone_Project/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
#import libraries
import os
from pathlib import Path
import pandas as pd
from PIL import Image
import numpy as np
import cv2
from skimage.filters import threshold_otsu
from skimage import morphology

In [27]:
from google.colab import drive
drive.mount('/content/drive') # Mount to the existing '/content/drive' directory

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
# point to your mounted drive folder in Colab
base_dir = Path('/content/drive/MyDrive/ProjectMaterials/dataset')

#defining the exact image & mask subfolders
images_dir = base_dir / 'HAM10000_images'
masks_dir  = (base_dir / 'HAM10000_masks' / 'HAM10000_segmentations_lesion_tschandl')

#loading metadata
metadata_df = pd.read_csv(base / 'HAM10000_metadata.csv')


In [35]:
!pip install opencv-python scikit-image tensorflow matplotlib pandas


In [36]:
meta_path = base_dir / 'HAM10000_metadata.csv'
meta_data_df = pd.read_csv(meta_path)

In [37]:
 #– images are named e.g. ISIC_0000000.jpg
#    – masks are named e.g. ISIC_0000000_segmentation.png
meta_data_df['image_path'] = meta_data_df['image_id'].apply(lambda x: str(images_dir / f"{x}.jpg"))
meta_data_df['mask_path']  = meta_data_df['image_id'].apply(lambda x: str(masks_dir  / f"{x}_segmentation.png"))


In [39]:
# If any are missing, it probably means you typo’d a folder name or
# didn’t unzip something in the right place.
missing_imgs = [p for p in meta_data_df['image_path'] if not Path(p).exists()]
missing_masks = [p for p in meta_data_df['mask_path'] if not Path(p).exists()]

print(f"Total records: {len(meta_data_df)}")
print(f"Images missing: {len(missing_imgs)}")
print(f"Masks  missing: {len(missing_masks)}")


Total records: 10015
Images missing: 7705
Masks  missing: 10015
